# Preference Pair Generation Pipeline

## Objective

The goal of this pipeline is to generate high-quality preference pairs for Direct Preference Optimization (DPO) training, as required in Milestone 1 of the MNLP project. Each preference pair consists of two plausible answers (A and B) to a question, with one being preferred. The answers must be:
- Similar in quality (not obviously better/worse),
- Diverse in formulation,
- Sufficiently complex and informative.

---

## Overview of the Strategy

The 100 assigned questions are divided into four blocks of 25 questions, each block applying a different combination of prompting strategies to generate two responses (A and B). This setup enables controlled comparisons between prompting techniques.

---

## Block Breakdown

### Block 1: Zero-shot vs Zero-shot Chain-of-Thought
**Questions 1–25**

- **Answer A:** Generated with standard zero-shot prompting.  
- **Answer B:** Generated with zero-shot chain-of-thought (CoT) prompting (adds *“Let’s think step by step”*).

**Goal:** Evaluate how explicit reasoning improves clarity and alignment.

---

### Block 2: Few-shot vs Few-shot CoT
**Questions 26–50**

- **Answer A:** Generated with few-shot prompting (2–3 QA examples in prompt).  
- **Answer B:** Same examples as A, but with step-by-step reasoning (few-shot CoT).

**Goal:** Test the impact of reasoning + demonstrations on answer helpfulness.

---

### Block 3: One-turn vs Multi-turn Refinement
**Questions 51–75**

- **Answer A:** Generated in a single-shot prompt.  
- **Answer B:** Second turn: ask GPT to refine the first response (*“Improve your previous answer”*).

**Goal:** Observe whether iterative prompting leads to more informative or coherent responses.

---

### Block 4: Neutral vs Role Prompting
**Questions 76–100**

- **Answer A:** Standard neutral prompt.  
- **Answer B:** Role prompting (e.g. *“You are a scientist. Please answer rigorously”*).

**Goal:** Investigate the effect of model persona on clarity and precision.

---

## Prompt Execution and Automation

- Responses are generated using the `gpt_wrapper` interface.  
- For each question:
  - A dedicated chat is created.
  - Both A and B responses are generated with controlled prompts.

Each preference pair is stored in a structured format. 

Note: It is not a problem that some B answers begin with phrases like "To improve the previous answer...", "Certainly!", or “Let’s now clarify...” in the self-improve group; on the contrary, this underlines that the answer aims to refine the previous one, which is consistent with the objective of the DPO, i.e., to learn to prefer more complete and clear answers. As long as answer B is actually better reasoned than A, this introductory style does not introduce harmful bias, but makes the preference more explicit and instructive, because the rest of the answer is generally objectively better, and the preference is not based solely on the form but on the enriched content. It is simply necessary to ensure that the substantive quality of B justifies the preference, and is not based solely on the form (these are not obvious artifacts like "this is a bad answer").

---

## Code Implementation

In [ ]:
# Import needed libraries
import json
import copy
import os
import gpt_wrapper
from gpt_wrapper import api_base, api_key
from gpt_wrapper.chat import Chat

In [ ]:
# API Configuration
gpt_wrapper.api_base = "http://mnlp-backend-lb-1062233132.eu-central-1.elb.amazonaws.com"
gpt_wrapper.api_key = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"

In [ ]:
# Loading the original JSON file
input_path = "raw_data/XXXXXX.json"
with open(input_path, "r", encoding="utf-8") as f:
    original_data = json.load(f)

data = copy.deepcopy(original_data)

In [ ]:
# Function to format the question depending on whether it is an MCQ type or not
def format_question_with_options(question):
    body = question.get("question_body", "").strip()
    if question.get("question_type") == "mcq":
        options = question.get("question_options", [])
        options_text = "\n".join([f"{i + 1}. {opt}" for i, opt in enumerate(options)])
        return f"{body}\n{options_text}"
    return body

In [ ]:
# Prompting strategy function
def get_prompts(question, idx):
    base = format_question_with_options(question)
    
    if idx < 25:
        return base, base + " Let's think step by step."
    
    elif idx < 50:
        few_shot = (
            "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. "
            "Each can has 3 tennis balls. How many tennis balls does he have now?\n"
            "A: The answer is 11.\n"
            f"Q: {base}\nA:"
        )
        few_shot_cot = (
            "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. "
            "Each can has 3 tennis balls. How many tennis balls does he have now?\n"
            "A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. "
            "5 + 6 = 11. The answer is 11.\n"
            f"Q: {base}\nA:"
        )
        return few_shot, few_shot_cot
    
    elif idx < 75:
        return base, "Improve your previous answer"
    
    else:
        return base, f"You are a scientist. Please answer rigorously: {base}"

In [ ]:
# New data to store updated structure
updated_data = []

# Loop through each question
for idx, question in enumerate(data):
    qid = question.get("question_id")
    prompt_a, prompt_b = get_prompts(question, idx)

    try:
        if 50 <= idx < 75:
            chat = Chat.create(f"Q{qid}_AB")
            answer_a = chat.ask(content=prompt_a).to_dict()
            answer_b = chat.ask(content=prompt_b).to_dict()
            A_text = answer_a["content"].replace("Answer:", "").strip()
            B_text = answer_b["content"].replace("Answer:", "").strip()
            A_id = B_id = chat.chat_id
        else:
            chat_a = Chat.create(f"Q{qid}_A")
            answer_a = chat_a.ask(content=prompt_a).to_dict()
            chat_b = Chat.create(f"Q{qid}_B")
            answer_b = chat_b.ask(content=prompt_b).to_dict()
            A_text = answer_a["content"].replace("Answer:", "").strip()
            B_text = answer_b["content"].replace("Answer:", "").strip()
            A_id = chat_a.chat_id
            B_id = chat_b.chat_id

        # Preserve all original fields and reinsert fields in correct order
        updated_entry = {
            "course_id": question["course_id"],
            "question_id": question["question_id"],
            "question_body": question["question_body"],
            "question_options": question.get("question_options"),
            "question_answer": question["question_answer"],
            "question_type": question["question_type"],
            "A_chat_id": A_id,
            "B_chat_id": B_id,
            "A": A_text,
            "B": B_text,
            "ranking_criteria": question["ranking_criteria"]
        }

        updated_data.append(updated_entry)
        print(f"- Q{qid} processed")

    except Exception as e:
        print(f"- Q{qid} error: {e}")

In [ ]:
# Save to file
with open("XXXXXX.json", "w", encoding="utf-8") as f_out:
    json.dump(updated_data, f_out, indent=4, ensure_ascii=False)

Now that we have successfully generated and validated our 100 high-quality preference pairs, we can proceed with the following steps of the project.

Before using the data for training, we must carefully review each pair (A, B) to ensure that:
- Both are plausible and relevant.
- The preferred answer is truly better in terms of correctness, completeness, clarity, and reasoning.
- Manually fill the ranking criteria field for each sample: overall, correctness, clarity, relevance, completeness.
- Make sure each sample has a clear overall preference, either A or B.

Analysis of Prompting Strategies for Preference Pair Generation:

Block 1 (Zero-shot vs Zero-shot Chain-of-Thought CoT): The "Let's think step by step" strategy used in Zero-shot CoT prompting is particularly effective for open-ended questions. It encourages the model to articulate a clearer, more logical reasoning path, which improves the answer's structure and makes the rationale more transparent. This often leads to better alignment and informativeness. However, for simpler or more factual questions (especially in multiple-choice format), this additional reasoning can make the response unnecessarily long and sometimes even misleading. The added complexity occasionally confuses the model, leading it to incorrect conclusions that would not occur in the more concise zero-shot responses. In conclusiob, this strategy is powerful for complex reasoning tasks but can be counterproductive for straightforward questions.

Block 2 (Few-shot vs Few-shot CoT): We observed that the differences between answers generated via standard few-shot and CoT few-shot prompting were often minimal. This is likely because the example used in the paper for the few-shot prompt (“Roger has 5 tennis balls…”) was too simple compared to the academic and technical nature of the target questions. Since the model was asked to generalize from a basic arithmetic example to questions about algorithms, grammar, or logic, the benefit of CoT was diluted. Thus, this strategy could be improved by selecting or crafting more domain-relevant and complex few-shot examples that match the style and reasoning level of the dataset.

Block 3 (One-turn vs Multi-turn Refinement): This approach involved generating the first answer normally, and then asking the model to “improve” it in a second turn. We found that the second response was generally more detailed, structured, and complete. However, when the initial answer was incorrect, the model often retained the same underlying reasoning and simply improved the surface form (clarity, completness). It rarely revised the answer fundamentally, unless explicitly prompted to re-evaluate. Therefore, this strategy works well for refining expression, but not necessarily for correcting logical or factual errors. Prompting with “Can you double-check your reasoning?” or “Are you sure?” might yield better revisions.

Block 4 (Neutral vs Scientist Role Prompting): In this setup, the second responses were generated with role prompting: “You are a scientist. Please answer rigorously.” These responses were often more structured, clear, and used more technical vocabulary. However, in many cases, the actual content of the answer remained similar to the neutral version, just more formal in tone. A more effective variation could be to tailor the role (e.g., physicist, data scientist, linguist) based on the question topic. So, role prompting has clear benefits for clarity and tone but could be enhanced through domain-specific roles. 

In [ ]:
import json

# Load data from the original file
with open("XXXXXX.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Add the "other" key with an empty value in ranking_criteria if missing
for entry in data:
    if "ranking_criteria" in entry:
        if "other" not in entry["ranking_criteria"]:
            entry["ranking_criteria"]["other"] = ""

# Overwrite the original file with the updated data
with open("XXXXXX.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)